In [1]:
import numpy as np
import os

import random
import pandas as pd
import math
import matplotlib.pyplot as plt

import astropy.units as u

import astropy.constants as const

import scipy.interpolate, scipy.optimize

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *
from plot_binned_detection_1d2d_0526 import *

In [3]:
from scipy.stats import poisson

In [4]:

detectors = ['CJPL', 'Kamioka', 'LNGS', 'SURF', 'SNOlab']
nuclei_2d = 'Xenon'

In [5]:
det_pcles = ['CNO', 'pep']
bg_ideals_2d = ['nubb_only', 'all']
metallicities = ['high', 'low']

In [6]:
alphas_powers = [(0.001, 0.9), (0.01, 0.9), (0.1, 0.9),
                (0.001, 0.95), (0.01, 0.95), (0.1, 0.95),
                (0.001, 0.8), (0.01, 0.8), (0.1, 0.8)]
nubb_fractions = [0, 0.0001, 0.001, 0.01, 0.1, 1]
bg_ideals_2d = ['nubb_only', 'all']

In [7]:
result_pd = pd.DataFrame(index = [str(alpha_power) for alpha_power in alphas_powers],
                         columns=[det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)
                                  for det_pcle in det_pcles
                                  for bg_ideal_2d in bg_ideals_2d
                                  for detector in detectors 
                                  for  metallicity in metallicities
                                  for nubb_fraction in nubb_fractions])


In [8]:
result_pd

,CNO_nubb_only_CJPL_high_nubb0,CNO_nubb_only_CJPL_high_nubb0.0001,CNO_nubb_only_CJPL_high_nubb0.001,CNO_nubb_only_CJPL_high_nubb0.01,CNO_nubb_only_CJPL_high_nubb0.1,CNO_nubb_only_CJPL_high_nubb1,CNO_nubb_only_CJPL_low_nubb0,CNO_nubb_only_CJPL_low_nubb0.0001,CNO_nubb_only_CJPL_low_nubb0.001,CNO_nubb_only_CJPL_low_nubb0.01,...,pep_all_SNOlab_high_nubb0.001,pep_all_SNOlab_high_nubb0.01,pep_all_SNOlab_high_nubb0.1,pep_all_SNOlab_high_nubb1,pep_all_SNOlab_low_nubb0,pep_all_SNOlab_low_nubb0.0001,pep_all_SNOlab_low_nubb0.001,pep_all_SNOlab_low_nubb0.01,pep_all_SNOlab_low_nubb0.1,pep_all_SNOlab_low_nubb1
"(0.001, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.001, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.001, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
etaijk_pd = pd.read_csv('CNOpep'+'_Xe_ETAijk_regular_thrd0_S1S2binning.csv', index_col = 0)
etaijk_pd

,null_sum,alter_sum
CNO_nubb_only_CJPL_high_nubb0,473.408997,487.057326
CNO_nubb_only_CJPL_high_nubb0.0001,483.873022,497.521351
CNO_nubb_only_CJPL_high_nubb0.001,578.049252,591.697581
CNO_nubb_only_CJPL_high_nubb0.01,1519.811552,1533.459881
CNO_nubb_only_CJPL_high_nubb0.1,10937.434552,10951.082881
...,...,...
pep_all_SNOlab_low_nubb0.0001,2541.212014,2548.559757
pep_all_SNOlab_low_nubb0.001,2635.388244,2642.735987
pep_all_SNOlab_low_nubb0.01,3577.150544,3584.498287
pep_all_SNOlab_low_nubb0.1,12994.773543,13002.121286


In [10]:

exposures_groups = [
    [int(i) for i in np.logspace(np.log10(10), np.log10(1e4), 101)],
    [int(i) for i in np.logspace(np.log10(10), np.log10(1e4), 101)],
    [int(i) for i in np.logspace(np.log10(10), np.log10(1e4), 101)],
    [int(i) for i in np.logspace(np.log10(10), np.log10(1e5), 101)],
    [int(i) for i in np.logspace(np.log10(100), np.log10(1e5),101)],
    [int(i) for i in np.logspace(np.log10(1000), np.log10(1e5),101)]
                    ]

for bg, bg_ideal_2d in enumerate(bg_ideals_2d):   
    for detector in detectors:
        for det_pcle in det_pcles:
            for m, metallicity in enumerate(metallicities):
                for n, (nubb_fraction, exposures) in enumerate(zip(nubb_fractions, exposures_groups)):
                    
                    null_sum_ijkappa = etaijk_pd['null_sum'][det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)]
                    alter_sum_ijkappa =etaijk_pd['alter_sum'][det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)]
                    print(exposures)
                    for alpha_power in alphas_powers:
                        alpha, power = alpha_power
        
                        DT,_,_,_,_ = get_nume_DT_alphabeta_G_method2(exposures, 
                                        null_sum_ijkappa, alter_sum_ijkappa, alpha, power, simnum = 500000, 
                                 plot= False, print_check = False)
                      
                        result_pd[det_pcle + '_'+bg_ideal_2d+'_'+detector+'_'+metallicity+'_nubb'+str(nubb_fraction)][str(
                            alpha_power)] = DT
                        print(bg_ideal_2d, detector, det_pcle, metallicity, nubb_fraction,
                              'alpha = ', alpha, 'power = ', power, 'DT = ',DT, '\n')

           

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 891, 954, 1023, 1096, 1174, 1258, 1348, 1445, 1548, 1659, 1778, 1905, 2041, 2187, 2344, 2511, 2691, 2884, 3090, 3311, 3548, 3801, 4073, 4365, 4677, 5011, 5370, 5754, 6165, 6606, 7079, 7585, 8128, 8709, 9332, 10000]
nubb_only CJPL CNO high 0 alpha =  0.001 power =  0.9 DT =  49.40256839754328 

nubb_only CJPL CNO high 0 alpha =  0.01 power =  0.9 DT =  33.55386178861789 

nubb_only CJPL CNO high 0 alpha =  0.1 power =  0.9 DT =  16.96459930313589 

nubb_only CJPL CNO high 0 alpha =  0.001 power =  0.95 DT =  57.9671659676453 

nubb_only CJPL CNO high 0 alpha =  0.01 power =  0.95 DT =  40.818875872054754 

nubb_only CJPL CNO high 0 alpha =  0.1 power =  0.95 DT =  22.17910970907816 

nubb_only CJPL C

nubb_only CJPL CNO low 0 alpha =  0.001 power =  0.9 DT =  94.18447580645162 

nubb_only CJPL CNO low 0 alpha =  0.01 power =  0.9 DT =  64.0202877167097 

nubb_only CJPL CNO low 0 alpha =  0.1 power =  0.9 DT =  32.44926522043387 

nubb_only CJPL CNO low 0 alpha =  0.001 power =  0.95 DT =  110.51511735164496 

nubb_only CJPL CNO low 0 alpha =  0.01 power =  0.95 DT =  77.77138749101364 

nubb_only CJPL CNO low 0 alpha =  0.1 power =  0.95 DT =  42.077639046538025 

nubb_only CJPL CNO low 0 alpha =  0.001 power =  0.8 DT =  76.03490032558406 

nubb_only CJPL CNO low 0 alpha =  0.01 power =  0.8 DT =  49.475065924406685 

nubb_only CJPL CNO low 0 alpha =  0.1 power =  0.8 DT =  22.268257294857328 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 

nubb_only CJPL pep high 0 alpha =  0.001 power =  0.8 DT =  144.5475935828877 

nubb_only CJPL pep high 0 alpha =  0.01 power =  0.8 DT =  93.7143885928273 

nubb_only CJPL pep high 0 alpha =  0.1 power =  0.8 DT =  42.07317816851307 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 891, 954, 1023, 1096, 1174, 1258, 1348, 1445, 1548, 1659, 1778, 1905, 2041, 2187, 2344, 2511, 2691, 2884, 3090, 3311, 3548, 3801, 4073, 4365, 4677, 5011, 5370, 5754, 6165, 6606, 7079, 7585, 8128, 8709, 9332, 10000]
nubb_only CJPL pep high 0.0001 alpha =  0.001 power =  0.9 DT =  183.086142040325 

nubb_only CJPL pep high 0.0001 alpha =  0.01 power =  0.9 DT =  124.26634136423706 

nubb_only CJPL pep high 0.0001 alpha =  0.1 power =  0.9 DT =  62.90098872023395 

nubb_

nubb_only CJPL pep low 0.0001 alpha =  0.001 power =  0.9 DT =  169.35899443965855 

nubb_only CJPL pep low 0.0001 alpha =  0.01 power =  0.9 DT =  115.8725330436357 

nubb_only CJPL pep low 0.0001 alpha =  0.1 power =  0.9 DT =  58.55212038303694 

nubb_only CJPL pep low 0.0001 alpha =  0.001 power =  0.95 DT =  199.44727891156458 

nubb_only CJPL pep low 0.0001 alpha =  0.01 power =  0.95 DT =  140.14991293335441 

nubb_only CJPL pep low 0.0001 alpha =  0.1 power =  0.95 DT =  76.26124460874921 

nubb_only CJPL pep low 0.0001 alpha =  0.001 power =  0.8 DT =  136.84048537821377 

nubb_only CJPL pep low 0.0001 alpha =  0.01 power =  0.8 DT =  88.9296961777197 

nubb_only CJPL pep low 0.0001 alpha =  0.1 power =  0.8 DT =  40.16292735042735 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 

nubb_only Kamioka CNO high 0.0001 alpha =  0.1 power =  0.95 DT =  22.717972070431074 

nubb_only Kamioka CNO high 0.0001 alpha =  0.001 power =  0.8 DT =  40.7074578277597 

nubb_only Kamioka CNO high 0.0001 alpha =  0.01 power =  0.8 DT =  26.50749439395728 

nubb_only Kamioka CNO high 0.0001 alpha =  0.1 power =  0.8 DT =  11.905235960179345 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 891, 954, 1023, 1096, 1174, 1258, 1348, 1445, 1548, 1659, 1778, 1905, 2041, 2187, 2344, 2511, 2691, 2884, 3090, 3311, 3548, 3801, 4073, 4365, 4677, 5011, 5370, 5754, 6165, 6606, 7079, 7585, 8128, 8709, 9332, 10000]
nubb_only Kamioka CNO high 0.001 alpha =  0.001 power =  0.9 DT =  59.981447124304275 

nubb_only Kamioka CNO high 0.001 alpha =  0.01 power =  

...

all SURF pep high 0.0001 alpha =  0.001 power =  0.9 DT =  944.3361386138615 

all SURF pep high 0.0001 alpha =  0.01 power =  0.9 DT =  644.1716738197426 

all SURF pep high 0.0001 alpha =  0.1 power =  0.9 DT =  324.9862464948592 

all SURF pep high 0.0001 alpha =  0.001 power =  0.95 DT =  1109.5610878094978 

all SURF pep high 0.0001 alpha =  0.01 power =  0.95 DT =  780.2769552863763 

all SURF pep high 0.0001 alpha =  0.1 power =  0.95 DT =  423.3300198807156 

all SURF pep high 0.0001 alpha =  0.001 power =  0.8 DT =  764.6412244691096 

all SURF pep high 0.0001 alpha =  0.01 power =  0.8 DT =  496.46164745282414 

all SURF pep high 0.0001 alpha =  0.1 power =  0.8 DT =  222.84515579126816 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 

all SURF pep low 0.001 alpha =  0.001 power =  0.9 DT =  934.3368214519294 

all SURF pep low 0.001 alpha =  0.01 power =  0.9 DT =  638.0809694793536 

all SURF pep low 0.001 alpha =  0.1 power =  0.9 DT =  320.2845344345278 

all SURF pep low 0.001 alpha =  0.001 power =  0.95 DT =  1093.80277836245 

all SURF pep low 0.001 alpha =  0.01 power =  0.95 DT =  770.1972832296558 

all SURF pep low 0.001 alpha =  0.1 power =  0.95 DT =  418.9311395490552 

all SURF pep low 0.001 alpha =  0.001 power =  0.8 DT =  756.4860711859093 

all SURF pep low 0.001 alpha =  0.01 power =  0.8 DT =  492.01101350465456 

all SURF pep low 0.001 alpha =  0.1 power =  0.8 DT =  221.1559393997877 

[10, 10, 12, 13, 14, 15, 17, 19, 20, 22, 25, 27, 30, 33, 36, 39, 43, 47, 52, 57, 63, 69, 75, 83, 91, 100, 109, 120, 131, 144, 158, 173, 190, 208, 229, 251, 275, 301, 331, 363, 398, 436, 478, 524, 575, 630, 691, 758, 831, 912, 1000, 1096, 1202, 1318, 1445, 1584, 1737, 1905, 2089, 2290, 2511, 2754, 3019, 3311, 363

all SNOlab CNO high 0.01 alpha =  0.001 power =  0.9 DT =  369.6786042240588 

all SNOlab CNO high 0.01 alpha =  0.01 power =  0.9 DT =  250.82831814209015 

all SNOlab CNO high 0.01 alpha =  0.1 power =  0.9 DT =  126.90913015324843 

all SNOlab CNO high 0.01 alpha =  0.001 power =  0.95 DT =  433.08765915768845 

all SNOlab CNO high 0.01 alpha =  0.01 power =  0.95 DT =  305.0762529697928 

all SNOlab CNO high 0.01 alpha =  0.1 power =  0.95 DT =  165.4044032444959 

all SNOlab CNO high 0.01 alpha =  0.001 power =  0.8 DT =  298.54423696572076 

all SNOlab CNO high 0.01 alpha =  0.01 power =  0.8 DT =  194.2518135744808 

all SNOlab CNO high 0.01 alpha =  0.1 power =  0.8 DT =  87.05331787887569 

[100, 107, 114, 123, 131, 141, 151, 162, 173, 186, 199, 213, 229, 245, 263, 281, 301, 323, 346, 371, 398, 426, 457, 489, 524, 562, 602, 645, 691, 741, 794, 851, 912, 977, 1047, 1122, 1202, 1288, 1380, 1479, 1584, 1698, 1819, 1949, 2089, 2238, 2398, 2570, 2754, 2951, 3162, 3388, 3630, 3890, 

all SNOlab CNO low 0.1 alpha =  0.001 power =  0.9 DT =  2619.286492460725 

all SNOlab CNO low 0.1 alpha =  0.01 power =  0.9 DT =  1784.1321488552994 

all SNOlab CNO low 0.1 alpha =  0.1 power =  0.9 DT =  899.6511490168206 

all SNOlab CNO low 0.1 alpha =  0.001 power =  0.95 DT =  3079.849641189201 

all SNOlab CNO low 0.1 alpha =  0.01 power =  0.95 DT =  2166.017977851287 

all SNOlab CNO low 0.1 alpha =  0.1 power =  0.95 DT =  1176.9382716049379 

all SNOlab CNO low 0.1 alpha =  0.001 power =  0.8 DT =  2118.305499414956 

all SNOlab CNO low 0.1 alpha =  0.01 power =  0.8 DT =  1375.1622208040842 

all SNOlab CNO low 0.1 alpha =  0.1 power =  0.8 DT =  616.6630692038248 

[1000, 1047, 1096, 1148, 1202, 1258, 1318, 1380, 1445, 1513, 1584, 1659, 1737, 1819, 1905, 1995, 2089, 2187, 2290, 2398, 2511, 2630, 2754, 2884, 3019, 3162, 3311, 3467, 3630, 3801, 3981, 4168, 4365, 4570, 4786, 5011, 5248, 5495, 5754, 6025, 6309, 6606, 6918, 7244, 7585, 7943, 8317, 8709, 9120, 9549, 10000, 10

all SNOlab pep high 1 alpha =  0.001 power =  0.9 DT =  39526.2789171452 

all SNOlab pep high 1 alpha =  0.01 power =  0.9 DT =  26960.712279237934 

all SNOlab pep high 1 alpha =  0.1 power =  0.9 DT =  13589.662471825699 

all SNOlab pep high 1 alpha =  0.001 power =  0.95 DT =  46462.327966714896 

all SNOlab pep high 1 alpha =  0.01 power =  0.95 DT =  32684.796399751704 

all SNOlab pep high 1 alpha =  0.1 power =  0.95 DT =  17715.205864386062 

all SNOlab pep high 1 alpha =  0.001 power =  0.8 DT =  32033.50626020686 

all SNOlab pep high 1 alpha =  0.01 power =  0.8 DT =  20738.894283779868 

all SNOlab pep high 1 alpha =  0.1 power =  0.8 DT =  9338.050854191499 

[10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 28, 30, 32, 34, 37, 39, 42, 45, 48, 52, 56, 60, 64, 69, 74, 79, 85, 91, 97, 104, 112, 120, 128, 138, 147, 158, 169, 181, 194, 208, 223, 239, 257, 275, 295, 316, 338, 363, 389, 416, 446, 478, 512, 549, 588, 630, 676, 724, 776, 831, 891, 954, 1023, 1096, 117

In [12]:
result_pd

,CNO_nubb_only_CJPL_high_nubb0,CNO_nubb_only_CJPL_high_nubb0.0001,CNO_nubb_only_CJPL_high_nubb0.001,CNO_nubb_only_CJPL_high_nubb0.01,CNO_nubb_only_CJPL_high_nubb0.1,CNO_nubb_only_CJPL_high_nubb1,CNO_nubb_only_CJPL_low_nubb0,CNO_nubb_only_CJPL_low_nubb0.0001,CNO_nubb_only_CJPL_low_nubb0.001,CNO_nubb_only_CJPL_low_nubb0.01,...,pep_all_SNOlab_high_nubb0.001,pep_all_SNOlab_high_nubb0.01,pep_all_SNOlab_high_nubb0.1,pep_all_SNOlab_high_nubb1,pep_all_SNOlab_low_nubb0,pep_all_SNOlab_low_nubb0.0001,pep_all_SNOlab_low_nubb0.001,pep_all_SNOlab_low_nubb0.01,pep_all_SNOlab_low_nubb0.1,pep_all_SNOlab_low_nubb1
"(0.001, 0.9)",49.402568,50.486969,60.026391,156.863813,1123.532192,10794.373692,94.184476,96.014108,115.339812,305.31191,...,981.336582,1328.941785,4808.603683,39526.278917,897.499383,901.738693,935.34388,1271.603903,4609.782862,37897.254315
"(0.01, 0.9)",33.553862,34.354531,40.960048,106.905246,765.555244,7360.782184,64.020288,65.517135,78.455978,207.826576,...,668.120143,904.380464,3271.030445,26960.712279,611.021292,614.598209,636.807175,866.868322,3134.186364,25865.712159
"(0.1, 0.9)",16.964599,17.342311,20.672446,54.049911,386.302008,3707.66741,32.449265,33.09558,39.727142,104.955429,...,336.765165,458.212924,1653.940071,13589.662472,309.360064,309.895127,321.763805,436.59358,1585.013401,13060.092062
"(0.001, 0.95)",57.967166,59.07515,70.512626,184.707849,1319.619955,12702.288028,110.515117,112.488303,135.234837,358.034617,...,1150.098746,1561.847918,5641.599782,46462.327967,1054.200126,1059.85575,1094.807944,1493.756884,5416.092474,44522.375453
"(0.01, 0.95)",40.818876,41.622186,49.749869,129.296781,928.555285,8899.673973,77.771387,79.448235,95.43316,251.585075,...,807.003652,1095.221246,3974.542466,32684.7964,741.226948,744.956129,770.81383,1050.855074,3806.321062,31325.684591
"(0.1, 0.95)",22.17911,22.715065,27.069707,70.452547,503.76466,4840.107299,42.077639,43.415197,51.61754,137.417362,...,439.990436,597.190413,2147.6875,17715.205864,402.915466,404.781054,419.155026,569.43521,2059.241167,17011.855615
"(0.001, 0.8)",39.864575,40.785602,48.565845,126.868286,907.843053,8722.176167,76.0349,77.847703,93.18575,246.606259,...,792.12168,1074.385567,3881.41987,32033.50626,726.165483,730.002659,755.618667,1028.039754,3724.826708,30699.090299
"(0.01, 0.8)",25.813341,26.432313,31.615102,82.241164,590.050738,5669.342863,49.475066,50.534443,60.516583,160.141263,...,514.78262,696.666615,2522.123596,20738.894284,472.586949,472.933455,490.797882,666.191937,2420.660126,19879.924147
"(0.1, 0.8)",11.755681,11.933328,14.228678,37.147976,265.159047,2547.689936,22.268257,22.833031,27.197711,72.143892,...,231.426856,312.288797,1131.245185,9338.050854,212.02165,212.667266,220.456174,298.947218,1084.855829,8936.10418


In [25]:
result_pd

,CNO_nubb_only_CJPL_high_nubb0,CNO_nubb_only_CJPL_high_nubb0.0001,CNO_nubb_only_CJPL_high_nubb0.001,CNO_nubb_only_CJPL_high_nubb0.01,CNO_nubb_only_CJPL_high_nubb0.1,CNO_nubb_only_CJPL_high_nubb1,CNO_nubb_only_CJPL_low_nubb0,CNO_nubb_only_CJPL_low_nubb0.0001,CNO_nubb_only_CJPL_low_nubb0.001,CNO_nubb_only_CJPL_low_nubb0.01,...,pep_all_SNOlab_high_nubb0.001,pep_all_SNOlab_high_nubb0.01,pep_all_SNOlab_high_nubb0.1,pep_all_SNOlab_high_nubb1,pep_all_SNOlab_low_nubb0,pep_all_SNOlab_low_nubb0.0001,pep_all_SNOlab_low_nubb0.001,pep_all_SNOlab_low_nubb0.01,pep_all_SNOlab_low_nubb0.1,pep_all_SNOlab_low_nubb1
"(0.001, 0.9)",49.364657,50.443259,59.990061,157.044458,1124.40481,10800.572084,94.106519,96.14037,115.253052,304.90751,...,980.946532,1329.498978,4805.062112,39587.714856,897.876213,901.797386,936.681847,1272.584166,4614.035244,37909.93296
"(0.01, 0.9)",33.632181,34.393193,40.923305,106.99886,765.658061,7348.731003,63.887106,65.47867,78.372222,207.110814,...,668.796219,905.239248,3272.039256,26943.147807,610.931537,614.834715,637.47388,867.658124,3140.154986,25888.936056
"(0.1, 0.9)",16.98744,17.385264,20.639145,53.985477,386.433252,3707.3464,32.50208,33.127763,39.723436,105.149029,...,336.885635,456.545957,1651.247827,13619.072831,309.456999,310.351593,321.727075,436.016934,1581.584574,13044.126939


In [13]:
file_name =  'CNOpep'+'_Xe_DT_regular_thrd0_numeNa_S1S2binning.csv'
file_name


'CNOpep_Xe_DT_regular_thrd0_numeNa_S1S2binning.csv'

In [14]:
result_pd.to_csv(file_name)